In [1]:
# Import các thư viện cần thiết
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import class_weight

# Đường dẫn đến dữ liệu
train_dir = '/kaggle/input/ai-training-challenge-hutech-orange-classifier/old_oranges_data_1/old_oranges_data/train_set'  
test_dir = '/kaggle/input/ai-training-challenge-hutech-orange-classifier/old_oranges_data_1/old_oranges_data/test_set'   
# Tiền xử lý dữ liệu
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Tạo generator cho tập train và validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Tạo generator cho tập test
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Tính toán class weights để xử lý mất cân bằng dữ liệu
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Sử dụng EfficientNetB0 làm mô hình cơ sở
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Đóng băng các lớp của mô hình cơ sở
base_model.trainable = False

# Thêm các lớp tùy chỉnh
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Tạo mô hình cuối cùng
model = Model(inputs=base_model.input, outputs=predictions)

# Biên dịch mô hình
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Huấn luyện mô hình ban đầu
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,
    callbacks=[early_stopping, checkpoint, reduce_lr],
    class_weight=class_weights
)

# Đánh giá mô hình trên tập test
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy before Fine-Tuning: {test_acc:.4f}')

# Fine-tuning: Mở một số lớp cuối của EfficientNetB0
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Biên dịch lại mô hình với learning rate nhỏ hơn
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks cho fine-tuning
fine_tune_checkpoint = ModelCheckpoint('fine_tuned_model.keras', monitor='val_accuracy', save_best_only=True)

# Huấn luyện lại mô hình với fine-tuning
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,  # Số epochs cho fine-tuning
    callbacks=[early_stopping, fine_tune_checkpoint, reduce_lr],
    class_weight=class_weights
)

# Đánh giá mô hình sau fine-tuning trên tập test
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy after Fine-Tuning: {test_acc:.4f}')

Found 1761 images belonging to 2 classes.
Found 440 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


55/55 ━━━━━━━━━━━━━━━━━━━━ 67s 802ms/step - accuracy: 0.4821 - loss: 1.0408 - val_accuracy: 0.4135 - val_loss: 0.7076 - learning_rate: 1.0000e-04
Epoch 2/30
 1/55 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5312 - loss: 0.8454

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 95ms/step - accuracy: 0.5312 - loss: 0.8454 - val_accuracy: 0.5417 - val_loss: 0.6897 - learning_rate: 1.0000e-04
Epoch 3/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 444ms/step - accuracy: 0.4827 - loss: 0.9658 - val_accuracy: 0.4183 - val_loss: 0.7330 - learning_rate: 1.0000e-04
Epoch 4/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5625 - loss: 1.3469 - val_accuracy: 0.4583 - val_loss: 0.7196 - learning_rate: 1.0000e-04
Epoch 5/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 446ms/step - accuracy: 0.5058 - loss: 0.9257 - val_accuracy: 0.4135 - val_loss: 0.7426 - learning_rate: 1.0000e-04
Epoch 6/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7188 - loss: 0.5782 - val_accuracy: 0.5417 - val_loss: 0.6950 - learning_rate: 2.0000e-05
Epoch 7/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 27s 432ms/step - accuracy: 0.5119 - loss: 0.8986 - val_accuracy: 0.4255 - val_loss: 0.7683 - learning_rate: 2.0000e-05
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 513ms/step - accuracy: 0.2084 - loss: 0.7372
Te